# **Telco Churn - EDA**

**Descrição:** Notebook de Análise Exploratória (EDA) para o dataset *Telco Customer Churn*.  
**Objetivo:** inspecionar qualidade dos dados, gerar insights e preparar um dataset limpo para modelagem.  
> Observação: código e variáveis em *English*, explicações em *PT-BR*.

## **Imports e configurações iniciais**

In [94]:
# Bibliotecas

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## **Definir caminhos (pathlib)**

In [95]:
# Considerando que este notebook está em ml/notebooks/
PROJECT_ROOT = Path.cwd().parents[1]

RAW_DATA_DIR = PROJECT_ROOT / "ml" / "data" / "raw"
PROCESSED_DATA_DIR = PROJECT_ROOT / "ml" / "data" / "processed"
DATA_PATH = RAW_DATA_DIR / "WA_Fn-UseC_-Telco-Customer-Churn.csv"

#DATA_PATH

## **Carregar dados**

In [96]:
# Carrega o dataset
telecom_raw_df = pd.read_csv(DATA_PATH, low_memory=False)

## **Visão geral do dataset**

In [97]:
# Visualiza uma amostra dos dados
telecom_raw_df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1158,5334-JLAXU,Female,0,Yes,No,60,Yes,Yes,Fiber optic,Yes,...,No,No,No,Yes,Month-to-month,Yes,Credit card (automatic),94.10,5475.9,No
820,4220-TINQT,Female,0,Yes,No,61,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,106.35,6751.35,No
1436,9546-CQJSU,Female,0,No,No,2,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,91.40,193.6,Yes
2412,1623-NLDOT,Female,0,Yes,No,42,No,No phone service,DSL,No,...,No,Yes,No,No,One year,No,Mailed check,33.55,1445.3,Yes
6173,2378-HTWFW,Male,1,No,No,35,Yes,Yes,Fiber optic,Yes,...,No,No,No,Yes,Month-to-month,No,Credit card (automatic),91.00,3180.5,No


### **Shape e tipos**

In [98]:
# Dimensão do dataset
telecom_raw_df.shape

(7043, 21)

In [99]:
# Tipo de dados das colunas
telecom_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### **Valores nulos e duplicados**

In [100]:
# Valores nulos por coluna
telecom_raw_df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Aparente não há valores nulos ou duplicados no dataset. Mas é importante investigar mais a fundo. Será que há valores vazios representados como strings?

In [101]:
# Verifica a existência de valores vazios representados como strings
telecom_raw_df.select_dtypes(include=['object']).apply(lambda x: x.str.strip() == '').sum()

customerID           0
gender               0
Partner              0
Dependents           0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
TotalCharges        11
Churn                0
dtype: int64

In [102]:
# Olha os valores únicos da coluna 'TotalCharges'
telecom_raw_df['TotalCharges'].value_counts().reset_index().head(5)

,TotalCharges,count
0,,11
1,20.2,11
2,19.75,9
3,20.05,8
4,19.9,8


Na coluna `TotalCharges`, por exemplo, há valores vazios representados como strings.

In [103]:
# Valores duplicados
telecom_raw_df.duplicated().sum()

np.int64(0)

#### **Valores únicos por coluna**

In [104]:
# Valores unicos para as colunas do tipo object
telecom_raw_df.select_dtypes(include=['object']).nunique()

customerID          7043
gender                 2
Partner                2
Dependents             2
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
TotalCharges        6531
Churn                  2
dtype: int64

## **Limpeza inicial / normalização de strings**

In [105]:
# Cria uma cópia do dataset para evitar modificar o original
telecom_df = telecom_raw_df.copy()

In [106]:
# limpeza preliminar (strip strings)
for c in telecom_df.select_dtypes(include="object").columns:
    telecom_df[c] = telecom_df[c].astype(str).str.strip()      # evita None issues

## **Substituir strings vazias por NaN**

In [107]:
# Substitui valores em branco por np.nan
telecom_df['TotalCharges'] = telecom_df['TotalCharges'].replace('', np.nan)

# Verifica se a substituição foi feita corretamente
telecom_df['TotalCharges'].isnull().sum()


np.int64(11)

## **Renomear colunas**

In [108]:
# Mapping dos nomes das colunas (original -> português)

mapping = {
    "customerID": "id_cliente",
    "gender": "genero",
    "SeniorCitizen": "idoso",
    "Partner": "parceiro",
    "Dependents": "dependentes",
    "tenure": "tempo_contrato_meses",
    "PhoneService": "servico_telefone",
    "MultipleLines": "linhas_multiplas",
    "InternetService": "tipo_internet",
    "OnlineSecurity": "seguranca_online",
    "OnlineBackup": "backup_online",
    "DeviceProtection": "protecao_dispositivo",
    "TechSupport": "suporte_tecnico",
    "StreamingTV": "streaming_tv",
    "StreamingMovies": "streaming_filmes",
    "Contract": "tipo_contrato",
    "PaperlessBilling": "cobranca_digital",
    "PaymentMethod": "metodo_pagamento",
    "MonthlyCharges": "cobranca_mensal",
    "TotalCharges": "cobranca_total",
    "Churn": "churn"
}


In [109]:
# Aplicar rename e salvar mapping
telecom_df.rename(columns=mapping, inplace=True)

In [110]:
# Verifica a mudança de nomes
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_cliente            7043 non-null   object 
 1   genero                7043 non-null   object 
 2   idoso                 7043 non-null   int64  
 3   parceiro              7043 non-null   object 
 4   dependentes           7043 non-null   object 
 5   tempo_contrato_meses  7043 non-null   int64  
 6   servico_telefone      7043 non-null   object 
 7   linhas_multiplas      7043 non-null   object 
 8   tipo_internet         7043 non-null   object 
 9   seguranca_online      7043 non-null   object 
 10  backup_online         7043 non-null   object 
 11  protecao_dispositivo  7043 non-null   object 
 12  suporte_tecnico       7043 non-null   object 
 13  streaming_tv          7043 non-null   object 
 14  streaming_filmes      7043 non-null   object 
 15  tipo_contrato        

## **Padronização dos valores das variáveis**

In [111]:
def apply_maps(df, maps):
    for col, mp in maps.items():
        if col in df.columns:
            df[col] = df[col].replace(mp)
    return df

#### **Mapeamentos das variáveis categóricas**

In [112]:
category_value_maps = {
    "genero": {
        "Male": "homem",
        "Female": "mulher"
    },
    "linhas_multiplas": {
        "No phone service": "sem_telefone",
        "No": "nao",
        "Yes": "sim"
    },
    "tipo_internet": {
        "DSL": "dsl",
        "Fiber optic": "fibra",
        "No": "sem_internet"
    },
    "seguranca_online": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "backup_online": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "protecao_dispositivo": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "suporte_tecnico": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "streaming_tv": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "streaming_filmes": {
        "No internet service": "sem_internet",
        "No": "nao",
        "Yes": "sim"
    },
    "tipo_contrato": {
        "Month-to-month": "mensal",
        "One year": "um_ano",
        "Two year": "dois_anos"
    },
    "metodo_pagamento": {
        "Electronic check": "cheque_eletronico",
        "Mailed check": "cheque_enviado",
        "Bank transfer (automatic)": "transferencia_bancaria_automatica",
        "Credit card (automatic)": "cartao_credito_automatica"
    }
} 

# Aplica os mapeamentos de valores
telecom_df = apply_maps(telecom_df, category_value_maps)


#### **Mapeamentos das variáveis binárias**

In [113]:
binary_value_maps = {
    "idoso": {0: 0, 1: 1},  # opcional, só pra documentar
    "parceiro": {"Yes": 1, "No": 0},
    "dependentes": {"Yes": 1, "No": 0},
    "servico_telefone": {"Yes": 1, "No": 0},
    "cobranca_digital": {"Yes": 1, "No": 0}
}

# Aplica os mapeamentos de valores
telecom_df = apply_maps(telecom_df, binary_value_maps)


/tmp/ipykernel_224871/1083460203.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mp)


### **Codificação do target `churn`**

In [114]:
# Codificação do target `churn`
telecom_df["churn"] = telecom_df["churn"].map({"Yes": 1, "No": 0})

In [115]:
# Verifica a tradução
telecom_df.sample(5)

,id_cliente,genero,idoso,parceiro,dependentes,tempo_contrato_meses,servico_telefone,linhas_multiplas,tipo_internet,seguranca_online,...,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,cobranca_digital,metodo_pagamento,cobranca_mensal,cobranca_total,churn
7003,4501-VCPFK,homem,0,0,0,26,0,sem_telefone,dsl,nao,...,sim,sim,nao,nao,mensal,0,cheque_eletronico,35.75,1022.5,0
1065,0634-SZPQA,mulher,0,0,0,23,1,sim,fibra,sim,...,nao,nao,sim,nao,mensal,1,transferencia_bancaria_automatica,90.05,2169.8,1
2584,3969-JQABI,mulher,0,1,0,58,1,nao,dsl,sim,...,nao,nao,sim,nao,mensal,1,cartao_credito_automatica,65.25,3791.6,0
1588,8152-UOBNY,mulher,1,0,0,50,1,sim,fibra,nao,...,nao,sim,sim,sim,um_ano,1,cheque_eletronico,106.80,5347.95,0
3586,8722-NGNBH,homem,0,0,0,5,0,sem_telefone,dsl,sim,...,sim,sim,nao,nao,mensal,0,cheque_enviado,40.00,223.45,1


## **Conversão de tipos**

### **Colunas string → `str`**

In [116]:
str_cols = [
    "id_cliente"
]

telecom_df[str_cols] = telecom_df[str_cols].astype("string")

### **Colunas categóricas → `pd.Categorical`**

In [117]:
categorical_cols = [
    "genero",
    "linhas_multiplas",
    "tipo_internet",
    "seguranca_online",
    "backup_online",
    "protecao_dispositivo",
    "suporte_tecnico",
    "streaming_tv",
    "streaming_filmes",
    "tipo_contrato",
    "metodo_pagamento"
]

telecom_df[categorical_cols] = telecom_df[categorical_cols].astype("category")


### **Colunas numéricas → `float` / `int`**

In [118]:
float_cols = [
    "cobranca_mensal",
    "cobranca_total"
]

telecom_df[float_cols] = telecom_df[float_cols].astype("float32")

In [119]:
integer_cols = [
    "tempo_contrato_meses"
]

telecom_df[integer_cols] = telecom_df[integer_cols].astype("int16")

In [120]:
binary_cols = [
    "idoso",
    "parceiro",
    "dependentes",
    "servico_telefone",
    "cobranca_digital"
]

telecom_df[binary_cols] = telecom_df[binary_cols].astype("int8")

### **Target `churn` → `int`**

In [121]:
telecom_df["churn"] = telecom_df["churn"].astype("int8")

In [122]:
# Verifica o resultado final
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id_cliente            7043 non-null   string  
 1   genero                7043 non-null   category
 2   idoso                 7043 non-null   int8    
 3   parceiro              7043 non-null   int8    
 4   dependentes           7043 non-null   int8    
 5   tempo_contrato_meses  7043 non-null   int16   
 6   servico_telefone      7043 non-null   int8    
 7   linhas_multiplas      7043 non-null   category
 8   tipo_internet         7043 non-null   category
 9   seguranca_online      7043 non-null   category
 10  backup_online         7043 non-null   category
 11  protecao_dispositivo  7043 non-null   category
 12  suporte_tecnico       7043 non-null   category
 13  streaming_tv          7043 non-null   category
 14  streaming_filmes      7043 non-null   category
 15  tipo

In [123]:
# Verifica uma amostra dos dados
telecom_df.sample(5)

,id_cliente,genero,idoso,parceiro,dependentes,tempo_contrato_meses,servico_telefone,linhas_multiplas,tipo_internet,seguranca_online,...,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,cobranca_digital,metodo_pagamento,cobranca_mensal,cobranca_total,churn
5977,0780-XNZFN,homem,0,0,0,57,1,nao,sem_internet,sem_internet,...,sem_internet,sem_internet,sem_internet,sem_internet,dois_anos,0,transferencia_bancaria_automatica,20.650000,1125.599976,0
5830,6754-WKSHP,homem,0,0,1,30,1,sim,sem_internet,sem_internet,...,sem_internet,sem_internet,sem_internet,sem_internet,um_ano,0,transferencia_bancaria_automatica,25.350000,723.299988,0
5301,2082-OJVTK,homem,0,1,1,29,1,sim,fibra,nao,...,sim,nao,nao,sim,mensal,1,cheque_eletronico,89.199997,2698.350098,1
4790,5216-WASFJ,mulher,1,1,0,31,1,sim,fibra,nao,...,nao,nao,nao,sim,mensal,1,cheque_eletronico,84.849998,2633.399902,0
5738,5049-GLYVG,homem,0,0,0,1,1,nao,sem_internet,sem_internet,...,sem_internet,sem_internet,sem_internet,sem_internet,mensal,0,cheque_enviado,20.600000,20.600000,1


## **Schema final dos dados**

## **Estatísticas descritivas**

### **Categóricas (summary)**

### **Numéricas (summary)**

## **Distribuição da variável alvo (Churn)**

## **Análise univariada - variáveis categóricas**

## **Análise univariada - variáveis numéricas**

## **Análise bivariada - categóricas vs Churn**

## **Testes estatísticos para categóricas**

## **Medida da força da associação (Cramér's V)**

## **Análise bivariada - numéricas vs Churn**

## **Matriz de correlação e multicolinearidade**

## **Informação mútua (Mutual Information)**

## **Outliers e inconsistências**

## **Feature engineering rápida**

## **Salvando amostra processada / artefatos**

## **Resumo de insights**